# Chapter 15

In [1]:
%use s2

import dev.nm.algebra.linear.matrix.doubles.operation.MatrixFactory.cbind
import dev.nm.stat.cointegration.JohansenAsymptoticDistribution.Test
import java.util.Arrays

println("Chapter 15 demos")

Chapter 15 demos


In [2]:
println("univariate time series")

// construct a time series
var ts1 = SimpleTimeSeries(doubleArrayOf(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0))
println("ts1:" + ts1)

univariate time series
ts1:length = 10
[1,] 1.000000, [2,] 2.000000, [3,] 3.000000, [4,] 4.000000, [5,] 5.000000, [6,] 6.000000, [7,] 7.000000, [8,] 8.000000, [9,] 9.000000, [10,] 10.000000, 


In [3]:
// construct a time series by taking the first difference of another one
// each term is the difference between two successive terms in the original time series
val ts2 = ts1.diff(1)
println("ts2:" + ts2)

ts2:length = 9
[1,] 1.000000, [2,] 1.000000, [3,] 1.000000, [4,] 1.000000, [5,] 1.000000, [6,] 1.000000, [7,] 1.000000, [8,] 1.000000, [9,] 1.000000, 


In [4]:
// construct a time series by dropping the first two terms of another one
val ts3 = ts1.drop(2)
println("ts3:" + ts3)

ts3:length = 8
[1,] 3.000000, [2,] 4.000000, [3,] 5.000000, [4,] 6.000000, [5,] 7.000000, [6,] 8.000000, [7,] 9.000000, [8,] 10.000000, 


In [5]:
// construct a time series by lagging another one
// This operation makes sense only for equi-distant data points.
val ts4 = ts1.lag(2)
println("ts4:" + ts4)

ts4:length = 8
[1,] 1.000000, [2,] 2.000000, [3,] 3.000000, [4,] 4.000000, [5,] 5.000000, [6,] 6.000000, [7,] 7.000000, [8,] 8.000000, 


In [6]:
println("univariate time series of S&P 500 daily close")

// read the daily S&P 500 data from a csv file
val spx_arr = DoubleUtils.readCSV2d("./resources/sp500_daily.csv", true, true)
// convert the csv file into a matrix for manipulation
val spx_M: Matrix = DenseMatrix(spx_arr)
// extract the column of adjusted close prices
val spx_v: Vector = spx_M.getColumn(5) // adjusted closes
// construct a univariate time series from the data
val spx = SimpleTimeSeries(spx_v.toArray())
println(spx)

univariate time series of S&P 500 daily close


./resources/sp500_daily.csv (No such file or directory)
java.io.FileNotFoundException: ./resources/sp500_daily.csv (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:219)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:112)
	at dev.nm.number.DoubleUtils.readCSV2d(wb:1605)
	at dev.nm.number.DoubleUtils.readCSV2d(wb:1591)
	at Line_9.<init>(Line_9.jupyter-kts:4)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.Bas

In [7]:
// compute simple returns from prices
val diff: DoubleArray = spx.diff(1).toArray()
val p0: DoubleArray = spx.lag(1).toArray()
var simple_returns: DoubleArray = DoubleArray(diff.size)
for (i in 0 until diff.size) {
    simple_returns[i] = diff[i] / p0[i]
}
println(Arrays.toString(simple_returns))

null
java.lang.NullPointerException
	at Line_10.<init>(Line_10.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [8]:
// compute log returns from prices
val p1: DoubleArray = spx.drop(1).toArray()
val p: DoubleArray = spx.toArray()
var log_returns: DoubleArray = DoubleArray(p1.size)
for (i in 0 until p1.size) {
    log_returns[i] = log10(p1[i]) - log10(p[i])
}
println(Arrays.toString(log_returns))

null
java.lang.NullPointerException
	at Line_11.<init>(Line_11.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [9]:
println("auto-correlations for S&P 500 monthly close")

// read the monthly S&P 500 data from a csv file
val spx_arr = DoubleUtils.readCSV2d("./resources/sp500_monthly.csv", true, true)
// convert the csv file into a matrix for manipulation
val spx_M: Matrix = DenseMatrix(spx_arr)
// extract the column of adjusted close prices
val spx_v: Vector = spx_M.getColumn(5) // adjusted closes
// construct a univariate time series from the data
val spx = SimpleTimeSeries(spx_v.toArray())
//      println(spx)

// ACVF for lags 0, 1, 2, ..., 24
val acvf = SampleAutoCovariance(spx)
for (i in 0 until 25) {
    println(String.format("acvf(%d) = %f", i, acvf.evaluate(i)))
}

auto-correlations for S&P 500 monthly close


./resources/sp500_monthly.csv (No such file or directory)
java.io.FileNotFoundException: ./resources/sp500_monthly.csv (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:219)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:112)
	at dev.nm.number.DoubleUtils.readCSV2d(wb:1605)
	at dev.nm.number.DoubleUtils.readCSV2d(wb:1591)
	at Line_12.<init>(Line_12.jupyter-kts:4)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.j

In [10]:
// ACF for lags 0, 1, 2, ..., 24
val acf = SampleAutoCorrelation(spx)
for (i in 0 until 25) {
    println(String.format("acf(%d) = %f", i, acf.evaluate(i)))
}

null
java.lang.NullPointerException
	at Line_13.<init>(Line_13.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [11]:
// PACF for lags 1, 2, ..., 24
val pacf = SamplePartialAutoCorrelation(spx)
for (i in 1 until 25) {
    println(String.format("pacf(%d) = %f", i, pacf.evaluate(i)))
}

null
java.lang.NullPointerException
	at Line_14.<init>(Line_14.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [12]:
// run the Ljung-Box test
val lb_test = LjungBox(log_returns, 14, 0)
println(String.format("The null hypothesis is: %s%n", lb_test.getNullHypothesis()))
println(String.format("The alternative hypothesis is: %s%n", lb_test.getAlternativeHypothesis()))
println(String.format("The test statistic = %f%n", lb_test.statistics()))
println(String.format("The p-value = %f", lb_test.pValue()))

null
java.lang.NullPointerException
	at Line_15.<init>(Line_15.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [13]:
println("classical time series decomposition")

val rng = StandardNormalRNG()

// construct an additive model
val additive_model = AdditiveModel(
        // the trend
        doubleArrayOf(
            1.0, 3.0, 5.0, 7.0, 9.0, 11.0, 13.0, 15.0, 17.0, 19.0
        ),
        // the seasonal component
        doubleArrayOf(
            0.0, 1.0
        ),
        // the source of randomness
        rng
)
println(additive_model)

classical time series decomposition
length = 10
[1,] 3.210076, [2,] 4.207675, [3,] 4.354602, [4,] 7.398784, [5,] 8.174874, [6,] 11.812901, [7,] 13.941011, [8,] 16.839446, [9,] 16.638740, [10,] 22.670682, 


In [14]:
// construct a multiplicative model
val multiplicative_model = MultiplicativeModel(
        // the trend
        doubleArrayOf(
            1.0, 3.0, 5.0, 7.0, 9.0, 11.0, 13.0, 15.0, 17.0, 19.0
        ),
        // the seasonal component
        doubleArrayOf(
            -1.0, 1.0
        ),
        // the source of randomness
        rng
)
println(multiplicative_model)

length = 10
[1,] -0.023313, [2,] -1.530389, [3,] 6.557648, [4,] -4.018728, [5,] 17.793586, [6,] 12.330600, [7,] 21.304488, [8,] 8.549570, [9,] -22.265570, [10,] -8.626085, 


In [15]:
// decompose the S&P 500 monthly returns into a classical additive model
val spx_model = MADecomposition(
                spx.toArray(), // the SP 500 monthly returns
                12 // the lenght of seasonality
        )
println(String.format("The trend component: %s", Arrays.toString(spx_model.getTrend())))

null
java.lang.NullPointerException
	at Line_18.<init>(Line_18.jupyter-kts:3)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [16]:
println(String.format("The seasonal component: %s", Arrays.toString(spx_model.getSeasonal())))

null
java.lang.NullPointerException
	at Line_19.<init>(Line_19.jupyter-kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [17]:
println(String.format("The random component: %s", Arrays.toString(spx_model.getRandom())))

null
java.lang.NullPointerException
	at Line_20.<init>(Line_20.jupyter-kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [18]:
println("AR models")

// define an AR(1) model
val ar1 = ARModel(
        doubleArrayOf(0.6) // phi_1 = 0.6
)
val nLags = 10
// compute the autocovariance function for the model
val acvf1 = AutoCovariance(ar1)
for (i in 0 until nLags) {
    println(String.format("The acvf of the AR(1) model at lag%d = %f", i, acvf1.evaluate(i.toDouble())))
}

AR models
The acvf of the AR(1) model at lag0 = 1.562500
The acvf of the AR(1) model at lag1 = 0.937500
The acvf of the AR(1) model at lag2 = 0.562500
The acvf of the AR(1) model at lag3 = 0.337500
The acvf of the AR(1) model at lag4 = 0.202500
The acvf of the AR(1) model at lag5 = 0.121500
The acvf of the AR(1) model at lag6 = 0.072900
The acvf of the AR(1) model at lag7 = 0.043740
The acvf of the AR(1) model at lag8 = 0.026244
The acvf of the AR(1) model at lag9 = 0.015746


In [19]:
// compute the autocorrelation function for the model
val acf1 = AutoCorrelation(ar1, nLags)
for (i in 0 until nLags) {
    println(String.format("The acf of the AR(1) model at lag%d = %f", i, acf1.evaluate(i.toDouble())))
}

The acf of the AR(1) model at lag0 = 1.000000
The acf of the AR(1) model at lag1 = 0.600000
The acf of the AR(1) model at lag2 = 0.360000
The acf of the AR(1) model at lag3 = 0.216000
The acf of the AR(1) model at lag4 = 0.129600
The acf of the AR(1) model at lag5 = 0.077760
The acf of the AR(1) model at lag6 = 0.046656
The acf of the AR(1) model at lag7 = 0.027994
The acf of the AR(1) model at lag8 = 0.016796
The acf of the AR(1) model at lag9 = 0.010078


In [20]:
// define an AR(2) model
val ar2 = ARModel(
        doubleArrayOf(1.2, -0.35)
)
// compute the autocovariance function for the model
val acvf2 = AutoCovariance(ar2)
for (i in 1 until nLags) {
    println(String.format("The acvf of the AR(2) model at lag%d = %f", i, acvf2.evaluate(i.toDouble())))
}
println("")
// compute the autocorrelation function for the model
val acf2 = AutoCorrelation(ar2, 10)
for (i in 0 until nLags) {
    println(String.format("The acf of the AR(2) model at lag%d = %f", i, acf2.evaluate(i.toDouble())))
}

The acvf of the AR(2) model at lag1 = 4.826546
The acvf of the AR(2) model at lag2 = 3.891403
The acvf of the AR(2) model at lag3 = 2.980392
The acvf of the AR(2) model at lag4 = 2.214480
The acvf of the AR(2) model at lag5 = 1.614238
The acvf of the AR(2) model at lag6 = 1.162018
The acvf of the AR(2) model at lag7 = 0.829438
The acvf of the AR(2) model at lag8 = 0.588620
The acvf of the AR(2) model at lag9 = 0.416040

The acf of the AR(2) model at lag0 = 1.000000
The acf of the AR(2) model at lag1 = 0.888889
The acf of the AR(2) model at lag2 = 0.716667
The acf of the AR(2) model at lag3 = 0.548889
The acf of the AR(2) model at lag4 = 0.407833
The acf of the AR(2) model at lag5 = 0.297289
The acf of the AR(2) model at lag6 = 0.214005
The acf of the AR(2) model at lag7 = 0.152755
The acf of the AR(2) model at lag8 = 0.108404
The acf of the AR(2) model at lag9 = 0.076621


In [21]:
// create a random number generator from the AR model
val sim = ARIMASim(ar2)
sim.seed(1234567890L)

// generate a random time series
val T = 500 // length of the time series
var x: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // call the RNG to generate random numbers according to the specification
    x[i] = sim.nextDouble()
}
// determine the cutoff lag using partial autocorrelation
val acf2_hat = SamplePartialAutoCorrelation(SimpleTimeSeries(x))
for (i in 1 until 5) {
    println(String.format("The empirical pacf of a time series at lag%d: %f", i, acf2_hat.evaluate(i)))
}
// fit an AR(2) model using the data
val fit = ConditionalSumOfSquares(
                x,
                2, // the AR order
                0,
                0)
println(String.format("phi: %s", Arrays.toString(fit.getARMAModel().phi())))
println(String.format("var of error term: %s", fit.`var`()))

The empirical pacf of a time series at lag1: 0.870717
The empirical pacf of a time series at lag2: -0.361752
The empirical pacf of a time series at lag3: 0.006046
The empirical pacf of a time series at lag4: 0.050507
phi: [1.0, 1.1861775215396104, -0.3622433585852758]
var of error term: 1.0322854887339732


In [22]:
// make forecast
val forecast_ar2 = ARMAForecast(SimpleTimeSeries(x), ar2)
println("The forecasts for the AR(2) model are")
for (j in 0 until 10) {
    println(forecast_ar2.next())
}

The forecasts for the AR(2) model are
[501]: 0.355169 (1.000000)
[502]: 0.370225 (2.440000)
[503]: 0.319961 (3.628100)
[504]: 0.254374 (4.416644)
[505]: 0.193263 (4.884637)
[506]: 0.142884 (5.144859)
[507]: 0.103819 (5.283771)
[508]: 0.074574 (5.355976)
[509]: 0.053152 (5.392841)
[510]: 0.037681 (5.411433)


In [23]:
println("MA models")

// define an MA(1) model
val ma1 = MAModel(
        doubleArrayOf(0.8) // theta_1 = 0.8
)

val nLags = 10
// compute the autocovariance function for the model
val acvf1 = AutoCovariance(ma1)
for (i in 0 until nLags) {
    println(String.format("the acvf of the MA(1) model at lag%d = %f", i, acvf1.evaluate(i.toDouble())))
}
println("")
// compute the autocorrelation function for the model
val acf1 = AutoCorrelation(ma1, nLags)
for (i in 0 until nLags) {
    println(String.format("the acf of the MA(1) model at lag%d = %f", i, acf1.evaluate(i.toDouble())))
}

MA models
the acvf of the MA(1) model at lag0 = 1.640000
the acvf of the MA(1) model at lag1 = 0.800000
the acvf of the MA(1) model at lag2 = 0.000000
the acvf of the MA(1) model at lag3 = 0.000000
the acvf of the MA(1) model at lag4 = 0.000000
the acvf of the MA(1) model at lag5 = 0.000000
the acvf of the MA(1) model at lag6 = 0.000000
the acvf of the MA(1) model at lag7 = 0.000000
the acvf of the MA(1) model at lag8 = 0.000000
the acvf of the MA(1) model at lag9 = 0.000000

the acf of the MA(1) model at lag0 = 1.000000
the acf of the MA(1) model at lag1 = 0.487805
the acf of the MA(1) model at lag2 = 0.000000
the acf of the MA(1) model at lag3 = 0.000000
the acf of the MA(1) model at lag4 = 0.000000
the acf of the MA(1) model at lag5 = 0.000000
the acf of the MA(1) model at lag6 = 0.000000
the acf of the MA(1) model at lag7 = 0.000000
the acf of the MA(1) model at lag8 = 0.000000
the acf of the MA(1) model at lag9 = 0.000000


In [24]:
// define an MA(2) model
val ma2 = MAModel(
        doubleArrayOf(-0.2, 0.01) // the moving-average coefficients
)
val acvf2 = AutoCovariance(ma2)
for (i in 1 until 10) {
    println(String.format("The acvf of the MA(2) model at lag%d: %f", i, acvf2.evaluate(i.toDouble())))
}
val acf2 = AutoCorrelation(ma2, 10)
for (i in 0 until 10) {
    println(String.format("The acf of the MA(2) model at lag%d: %f", i, acf2.evaluate(i.toDouble())))
}

The acvf of the MA(2) model at lag1: -0.202000
The acvf of the MA(2) model at lag2: 0.010000
The acvf of the MA(2) model at lag3: 0.000000
The acvf of the MA(2) model at lag4: 0.000000
The acvf of the MA(2) model at lag5: 0.000000
The acvf of the MA(2) model at lag6: 0.000000
The acvf of the MA(2) model at lag7: 0.000000
The acvf of the MA(2) model at lag8: 0.000000
The acvf of the MA(2) model at lag9: 0.000000
The acf of the MA(2) model at lag0: 1.000000
The acf of the MA(2) model at lag1: -0.194212
The acf of the MA(2) model at lag2: 0.009614
The acf of the MA(2) model at lag3: 0.000000
The acf of the MA(2) model at lag4: 0.000000
The acf of the MA(2) model at lag5: 0.000000
The acf of the MA(2) model at lag6: 0.000000
The acf of the MA(2) model at lag7: 0.000000
The acf of the MA(2) model at lag8: 0.000000
The acf of the MA(2) model at lag9: 0.000000


In [25]:
// define an MA(2) model
val ma3 = MAModel(
        0.1, // the mean
        doubleArrayOf(-0.5, 0.01), // the moving-average coefficients
        10.0 // the standard deviation
)

// create a random number generator from the MA model
val sim = ARIMASim(ma3)
sim.seed(1234567890L)

// generate a random time series
val T = 500 // length of the time series
var x: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // call the RNG to generate random numbers according to the specification
    x[i] = sim.nextDouble()
}

// determine the cutoff lag using autocorrelation
val acf3_hat = SampleAutoCorrelation(SimpleTimeSeries(x))
for (i in 0 until 5) {
    println(String.format("The empirical acf of a time series at lag%d: %f", i, acf3_hat.evaluate(i)))
}
// fit an MA(2) model using the data
val fit = ConditionalSumOfSquares(
                x,
                0,
                0,
                2) // the MA order
println(String.format("Theta: %s", Arrays.toString(fit.getARMAModel().theta())))
println(String.format("Var of error term: %s", fit.`var`()))

The empirical acf of a time series at lag0: 1.000000
The empirical acf of a time series at lag1: -0.405272
The empirical acf of a time series at lag2: 0.035492
The empirical acf of a time series at lag3: -0.100347
The empirical acf of a time series at lag4: 0.072677
Theta: [1.0, -0.5136330232597168, -0.024493176415614604]
Var of error term: 103.2830291835046


In [26]:
// make forecast
val forecast_ma3 = ARMAForecast(SimpleTimeSeries(x), ma3)
println("The forecasts for the MA(2) model are")
for (j in 0 until 10) {
    println(forecast_ma3.next())
}

The forecasts for the MA(2) model are
[501]: -2.246799 (1.000000)
[502]: 0.147899 (1.250000)
[503]: 0.100000 (1.250100)
[504]: 0.100000 (1.250100)
[505]: 0.100000 (1.250100)
[506]: 0.100000 (1.250100)
[507]: 0.100000 (1.250100)
[508]: 0.100000 (1.250100)
[509]: 0.100000 (1.250100)
[510]: 0.100000 (1.250100)


In [27]:
println("ARMA(1,1) model")

// build an ARMA(1, 1)
val arma11 = ARMAModel(
        doubleArrayOf(0.2), // the AR coefficients
        doubleArrayOf(1.1) // the MA coefficients
)
// compute the autocovariance function for the model
val acvf = AutoCovariance(arma11)
for (i in 1 until 10) {
    println(String.format("The acvf of the ARMA(1,1) model at lag%d: %f", i, acvf.evaluate(i.toDouble())))
}
// compute the autocorrelation function for the model
val acf = AutoCorrelation(arma11, 10)
for (i in 1 until 10) {
    println(String.format("The acf of the ARMA(1,1) model at lag%d: %f", i, acf.evaluate(i.toDouble())))
}

ARMA(1,1) model
The acvf of the ARMA(1,1) model at lag1: 1.652083
The acvf of the ARMA(1,1) model at lag2: 0.330417
The acvf of the ARMA(1,1) model at lag3: 0.066083
The acvf of the ARMA(1,1) model at lag4: 0.013217
The acvf of the ARMA(1,1) model at lag5: 0.002643
The acvf of the ARMA(1,1) model at lag6: 0.000529
The acvf of the ARMA(1,1) model at lag7: 0.000106
The acvf of the ARMA(1,1) model at lag8: 0.000021
The acvf of the ARMA(1,1) model at lag9: 0.000004
The acf of the ARMA(1,1) model at lag1: 0.598491
The acf of the ARMA(1,1) model at lag2: 0.119698
The acf of the ARMA(1,1) model at lag3: 0.023940
The acf of the ARMA(1,1) model at lag4: 0.004788
The acf of the ARMA(1,1) model at lag5: 0.000958
The acf of the ARMA(1,1) model at lag6: 0.000192
The acf of the ARMA(1,1) model at lag7: 0.000038
The acf of the ARMA(1,1) model at lag8: 0.000008
The acf of the ARMA(1,1) model at lag9: 0.000002


In [28]:
// build an ARMA(1, 1)
val arma11 = ARMAModel(
        doubleArrayOf(0.2), // the AR coefficients
        doubleArrayOf(1.1) // the MA coefficients
)
// compute the linear representation
val ma = LinearRepresentation(arma11)
for (i in 1..20) {
    println(String.format("The coefficients of the linear representation at lag %d = %f", i, ma.AR(i)))
}

The coefficients of the linear representation at lag 1 = 1.300000
The coefficients of the linear representation at lag 2 = 0.260000
The coefficients of the linear representation at lag 3 = 0.052000
The coefficients of the linear representation at lag 4 = 0.010400
The coefficients of the linear representation at lag 5 = 0.002080
The coefficients of the linear representation at lag 6 = 0.000416
The coefficients of the linear representation at lag 7 = 0.000083
The coefficients of the linear representation at lag 8 = 0.000017
The coefficients of the linear representation at lag 9 = 0.000003
The coefficients of the linear representation at lag 10 = 0.000001
The coefficients of the linear representation at lag 11 = 0.000000
The coefficients of the linear representation at lag 12 = 0.000000
The coefficients of the linear representation at lag 13 = 0.000000
The coefficients of the linear representation at lag 14 = 0.000000
The coefficients of the linear representation at lag 15 = 0.000000
The 

In [29]:
println("ARMA(2,3) model")

// build an ARMA(2, 3)
val arma23 = ARMAModel(
        doubleArrayOf(0.6, -0.23), // the AR coefficients
        doubleArrayOf(0.1, 0.2, 0.4) // the MA coefficients
)

// compute the linear representation
val ma = LinearRepresentation(arma23)
for (i in 1..20) {
    println(String.format("The coefficients of the linear representation at lag %d = %f", i, ma.AR(i)))
}

ARMA(2,3) model
The coefficients of the linear representation at lag 1 = 0.700000
The coefficients of the linear representation at lag 2 = 0.390000
The coefficients of the linear representation at lag 3 = 0.473000
The coefficients of the linear representation at lag 4 = 0.194100
The coefficients of the linear representation at lag 5 = 0.007670
The coefficients of the linear representation at lag 6 = -0.040041
The coefficients of the linear representation at lag 7 = -0.025789
The coefficients of the linear representation at lag 8 = -0.006264
The coefficients of the linear representation at lag 9 = 0.002173
The coefficients of the linear representation at lag 10 = 0.002745
The coefficients of the linear representation at lag 11 = 0.001147
The coefficients of the linear representation at lag 12 = 0.000057
The coefficients of the linear representation at lag 13 = -0.000230
The coefficients of the linear representation at lag 14 = -0.000151
The coefficients of the linear representation at l

In [30]:
// compute the autocovariance function for the model
val acvf = AutoCovariance(arma23)
for (i in 1 until 10) {
    println(String.format("The acvf of the ARMA(2,3) model at lag%d: %f", i, acvf.evaluate(i.toDouble())))
}

The acvf of the ARMA(2,3) model at lag1: 1.251651
The acvf of the ARMA(2,3) model at lag2: 0.792637
The acvf of the ARMA(2,3) model at lag3: 0.587703
The acvf of the ARMA(2,3) model at lag4: 0.170315
The acvf of the ARMA(2,3) model at lag5: -0.032983
The acvf of the ARMA(2,3) model at lag6: -0.058962
The acvf of the ARMA(2,3) model at lag7: -0.027791
The acvf of the ARMA(2,3) model at lag8: -0.003113
The acvf of the ARMA(2,3) model at lag9: 0.004524


In [31]:
// compute the autocorrelation function for the model
val acf = AutoCorrelation(arma23, 10)
for (i in 1 until 10) {
    println(String.format("The acf of the ARMA(2,3) model at lag%d: %f", i, acf.evaluate(i.toDouble())))
}

The acf of the ARMA(2,3) model at lag1: 0.656730
The acf of the ARMA(2,3) model at lag2: 0.415890
The acf of the ARMA(2,3) model at lag3: 0.308362
The acf of the ARMA(2,3) model at lag4: 0.089363
The acf of the ARMA(2,3) model at lag5: -0.017306
The acf of the ARMA(2,3) model at lag6: -0.030937
The acf of the ARMA(2,3) model at lag7: -0.014582
The acf of the ARMA(2,3) model at lag8: -0.001634
The acf of the ARMA(2,3) model at lag9: 0.002374


In [32]:
println("making forecasts for an ARMA model")

// define an ARMA(1, 1)
val arma11 = ARMAModel(
        doubleArrayOf(0.2), // the AR coefficients
        doubleArrayOf(1.1) // the MA coefficients
)

// create a random number generator from the ARMA model
val sim = ARIMASim(arma11)
sim.seed(1234567890L)

// generate a random time series
val T = 50 // length of the time series
var x: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // call the RNG to generate random numbers according to the specification
    x[i] = sim.nextDouble()
}

// compute the one-step conditional expectations for the model
val forecaset = ARMAForecastOneStep(x, arma11)
var x_hat: DoubleArray = DoubleArray(T)
var residuals: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // the one-step conditional expectations
    x_hat[i] = forecaset.xHat(i)
    // the errors
    residuals[i] = x[i] - x_hat[i]
}

making forecasts for an ARMA model


In [33]:
println(Arrays.toString(x))

[0.1968790947035005, 0.16584033866577874, 0.7910121229310072, 0.10657769691646068, -1.0371447908191465, 0.04412684959211502, 0.7241722424723148, -1.7538519814924725, -4.772738896937248, -2.7244783039780023, -1.1110789790665505, -2.106322783331434, -1.1271763863477884, 0.9965503611140754, 1.6572660541680069, 0.33320306655718857, -1.1870211371405786, -1.455695953473766, -1.1983843155249563, -0.6175694949485292, -1.638887243286174, -1.4684693838686553, 0.27241367827489105, -0.3568959657961215, -1.9094912681123621, -1.7414682817620857, 0.002962991657699643, 3.3215140309763758, 3.9748770865118255, 1.892138226291777, 0.24072858411567088, 0.579438081971123, 0.47801729694314576, -2.348962731013112, -3.5088899884329416, -2.2319690474266145, -0.11388062914051986, 0.7027802982941478, 0.9455377482452896, -0.37823117642937193, -0.03438897732456425, 1.0013070243160762, 1.1764996183304488, 2.655041457769469, 1.704878446908042, -0.455996185972092, -1.9281518997430394, -3.170380648633762, -3.0931897346

In [34]:
println(Arrays.toString(x_hat))

[0.0, 0.11783028083009503, 0.06297686421253082, 0.6826462678185927, -0.42568727132717654, -0.7032843656741394, 0.632629314138487, 0.22277838517806908, -2.058229893163323, -3.3247441761833425, -0.016461199464652077, -1.1919709121238073, -1.2353528600412458, -0.1287408986154927, 1.2083288982851947, 0.735022179437633, -0.295310396854917, -1.0419722536064209, -0.6649353913566711, -0.7221747128610998, -0.02881456074003297, -1.7864558441074792, -0.005432989589572779, 0.30648150332280744, -0.6732894222805461, -1.5039341530299661, -0.5639508511348783, 0.5154113730922484, 3.213026896542926, 1.4870556694784516, 0.7464601785514215, -0.41137892647394775, 1.016255913260322, -0.39353632813498596, -2.246948501083488, -1.8487285668630986, -0.7947267534662068, 0.5960758174822746, 0.23754730179573447, 0.8326648750944147, -1.1763613927854226, 1.0312014516375774, 0.17308632920739364, 1.1474473206254265, 1.9014906764895478, 0.16224352186193292, -0.6532191637766533, -1.5446325796392095, -2.1119998757720535,

In [35]:
println(Arrays.toString(residuals))

[0.1968790947035005, 0.04801005783568371, 0.7280352587184764, -0.576068570902132, -0.6114575194919699, 0.7474112152662544, 0.09154292833382782, -1.9766303666705416, -2.7145090037739252, 0.6002658722053402, -1.0946177796018985, -0.9143518712076266, 0.1081764736934574, 1.1252912597295681, 0.44893715588281213, -0.4018191128804445, -0.8917107402856616, -0.41372369986734503, -0.5334489241682852, 0.1046052179125706, -1.610072682546141, 0.3179864602388238, 0.2778466678644638, -0.6633774691189289, -1.236201845831816, -0.23753412873211954, 0.566913842792578, 2.8061026578841273, 0.7618501899688996, 0.4050825568133254, -0.5057315944357507, 0.9908170084450707, -0.5382386163171762, -1.955426402878126, -1.2619414873494534, -0.3832404805635159, 0.6808461243256869, 0.10670448081187323, 0.7079904464495551, -1.2108960515237865, 1.1419724154608584, -0.02989442732150116, 1.0034132891230552, 1.5075941371440427, -0.19661222958150582, -0.6182397078340249, -1.274932735966386, -1.6257480689945525, -0.981189858

In [36]:
println("fit an ARMA(1,1) time series")

// define an ARMA(1, 1)
val arma11 = ARMAModel(
        doubleArrayOf(0.2), // the AR coefficients
        doubleArrayOf(1.1) // the MA coefficients
)

// create a random number generator from the ARMA model
val sim = ARIMASim(arma11)
sim.seed(1234567890L)

// generate a random time series
val T = 50 // length of the time series
var x: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // call the RNG to generate random numbers according to the specification
    x[i] = sim.nextDouble()
}

// fit an ARMA(1,1) model
val css = ConditionalSumOfSquares(x, 1, 0, 1)
val arma11_css = css.getARMAModel()
println(String.format("The fitted ARMA(1,1) model is %s%n", arma11_css))
println(String.format("The fitted ARMA(1,1) model, demeanded, is %s%n", arma11_css.getDemeanedModel()))
println(String.format("AIC = %f", css.AIC()))

fit an ARMA(1,1) time series
The fitted ARMA(1,1) model is x_t = -0.25855664657157695 + (0.4087316816737802*x_{t-1}) + (1.0193696854247496*e_{t-1}) + e_t; var(e_t) = 0.886664, d = 0

The fitted ARMA(1,1) model, demeanded, is x_t = 0.0 + (0.4087316816737802*x_{t-1}) + (1.0193696854247496*e_{t-1}) + e_t; var(e_t) = 1.000000, d = 0

AIC = 93.971130


In [37]:
// compute the fitted values using the fitted model
val forecaset = ARMAForecastOneStep(x, arma11_css)
var x_hat: DoubleArray = DoubleArray(T)
var residuals: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // the fitted value
    x_hat[i] = forecaset.xHat(i)
    // the residuals
    residuals[i] = x[i] - x_hat[i]
}

println(Arrays.toString(x))
println(Arrays.toString(x_hat))
println(Arrays.toString(residuals))

[0.1968790947035005, 0.16584033866577874, 0.7910121229310072, 0.10657769691646068, -1.0371447908191465, 0.04412684959211502, 0.7241722424723148, -1.7538519814924725, -4.772738896937248, -2.7244783039780023, -1.1110789790665505, -2.106322783331434, -1.1271763863477884, 0.9965503611140754, 1.6572660541680069, 0.33320306655718857, -1.1870211371405786, -1.455695953473766, -1.1983843155249563, -0.6175694949485292, -1.638887243286174, -1.4684693838686553, 0.27241367827489105, -0.3568959657961215, -1.9094912681123621, -1.7414682817620857, 0.002962991657699643, 3.3215140309763758, 3.9748770865118255, 1.892138226291777, 0.24072858411567088, 0.579438081971123, 0.47801729694314576, -2.348962731013112, -3.5088899884329416, -2.2319690474266145, -0.11388062914051986, 0.7027802982941478, 0.9455377482452896, -0.37823117642937193, -0.03438897732456425, 1.0013070243160762, 1.1764996183304488, 2.655041457769469, 1.704878446908042, -0.455996185972092, -1.9281518997430394, -3.170380648633762, -3.0931897346

In [38]:
// run the goodness-of-fit test
val test = LjungBox(
        residuals,
        4, // check up to the 4-th lag
        2 // number of parameters
)
println(String.format("The test statistic = %f", test.statistics()))
println(String.format("The p-value = %f", test.pValue()))

The test statistic = 2.686421
The p-value = 0.261006


In [39]:
/**
*
* x<-c(0.2,0.3,-0.1,0.4,-0.5,0.6,0.1,0.2) adf.test(c(x,x),alternative =
* c("stationary"),4)
*
* The p-value from R is obtained form the interpolation
* (0.9-0.1)/(3.24-1.14)=(x-0.1)/(3.24-2.642)
* "x=0.8/(3.24-1.14)*(3.24-2.642)+0.1=0.3278"
*
* The values -3.24 and -1.14 are from table 4.2c in A. Banerjee, J. J.
* Dolado, J. W. Galbraith, and D. F. Hendry (1993): Cointegration, Error
* Correction, and the Econometric Analysis of Non-Stationary Data, Oxford
* University Press, Oxford.
*/

println("Unit root test using the augmented Dickey-Fuller test")

val sample: DoubleArray = doubleArrayOf(0.2, 0.3, -0.1, 0.4, -0.5, 0.6, 0.1, 0.2, 0.2, 0.3, -0.1, 0.4, -0.5, 0.6, 0.1, 0.2)
val adf = AugmentedDickeyFuller(
                sample,
                TrendType.CONSTANT, // constant drift term
                4, // the lag order
                null
        )

println("H0: " + adf.getNullHypothesis())
println("H1: " + adf.getAlternativeHypothesis())
println(String.format("\nThe p-value for the test = %f", adf.pValue()))
println(String.format("The statistics for the test = %f", adf.statistics()))

Unit root test using the augmented Dickey-Fuller test
H0: there is a unit root in the process, hence non-stationarity
H1: there is no unit root in the process, hence stationarity

The p-value for the test = 0.100000
The statistics for the test = -2.641969


In [40]:
val adf = AugmentedDickeyFuller(spx.toArray())
println(String.format("The p-value for the test = %f", adf.pValue()))
println(String.format("The statistics for the test = %f", adf.statistics()))

null
java.lang.NullPointerException
	at Line_43.<init>(Line_43.jupyter-kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [41]:
// auto fit an ARIMA model to S&P 500 prices
val fit_sp500_1 = AutoARIMAFit(spx.toArray(), 5, 1, 5, 0, 0, 1000)
val arima1 = fit_sp500_1.optimalModelByAIC()
val arima2 = fit_sp500_1.optimalModelByAICC()
println("The optimal model for S&P500 prices by AIC is:")
println(arima1)

null
java.lang.NullPointerException
	at Line_44.<init>(Line_44.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [42]:
// forecast prices based on the fitted model
val forecast_prices = ARIMAForecast(spx, arima1)
println("The next 10 price forecasts for sp500 using the ARIMA(2,1,1) model is:")
for (i in 0 until 10) {
    println(forecast_prices.next())
}

null
java.lang.NullPointerException
	at Line_45.<init>(Line_45.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [43]:
println("The optimal model for S&P500 prices by AICc is:")
println(arima2)

The optimal model for S&P500 prices by AICc is:


null
java.lang.NullPointerException
	at Line_46.<init>(Line_46.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [44]:
// auto fit an ARIMA model to S&P 500 log returns
val fit_sp500_2 = AutoARIMAFit(log_returns, 5, 1, 5, 0, 0, 1000)
val arima3 = fit_sp500_2.optimalModelByAIC()
val arima4 = fit_sp500_2.optimalModelByAICC()
println("The optimal model for S&P500 log returns by AIC is:")
println(arima3)

null
java.lang.NullPointerException
	at Line_47.<init>(Line_47.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [45]:
println("The optimal model for S&P500 log returns by AICc is:")
println(arima4)

The optimal model for S&P500 log returns by AICc is:


null
java.lang.NullPointerException
	at Line_48.<init>(Line_48.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [46]:
// forecast log returns based on the fitted model
val forecast_returns = ARIMAForecast(SimpleTimeSeries(log_returns), arima4)
println("The next 10 log return forecasts for sp500 using the ARMA(2, 3) model is:")
for (i in 0 until 10) {
    println(forecast_returns.next())
}

null
java.lang.NullPointerException
	at Line_49.<init>(Line_49.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [47]:
println("ARCH and GARCH models")

// σ_t^2 = 0.2 + 0.212 * ε_(t-1)^2
val arch1 = GARCHModel(
                0.2, // constant
                doubleArrayOf(0.212), // ARCH terms
                doubleArrayOf() // no GARCH terms
        )
println(arch1)
println(String.format("conditional variance = %f", arch1.`var`()))

ARCH and GARCH models
0.2+ 0.212000 * e_{t-1}
conditional variance = 0.253807


In [48]:
// σ_t^2= 0.2+0.212ε_(t-1)^2+0.106σ_(t-1)^2
val garch11 = GARCHModel(
                0.2, // constant
                doubleArrayOf(0.212), // ARCH terms
                doubleArrayOf(0.106) // GARCH terms
        )
println(garch11)
println(String.format("conditional variance = %f", garch11.`var`()))

0.2+ 0.212000 * e_{t-1}+ 0.106000 * h_{t-1}
conditional variance = 0.293255


In [49]:
// simulate the GARCH process
val sim = GARCHSim(garch11)
sim.seed(1234567890L)
val series: DoubleArray = RNGUtils.nextN(sim, 200)
println(Arrays.toString(series))

[0.09464235164181764, -0.04287154757437102, 0.4059365561386082, -0.505733772227959, 0.018704443199705746, 0.10203313531778042, 0.2290917436991943, -1.1774233639361313, -1.2612835802035602, 0.1202380988464397, -0.3805367043220213, -0.545565537829306, 0.2552633108201118, 0.34673063120199493, 0.3443670090107649, -0.3782080513709261, -0.21524174277686497, -0.3657868128866508, -0.040707517700922555, -0.13773868187169216, -0.5719439019603688, 0.09566942847716856, 0.17952130491984594, -0.3947771340704507, -0.4748601347528875, -0.17328133117367764, 0.34683413239310523, 1.2685821324311333, 0.3935412777861333, 0.28290224497354133, -0.35488649351970974, 0.6614933139819535, -0.6131010984348145, -0.7008491518045036, -0.9655537913618887, 0.1964457826647851, 0.0021557960459673863, 0.34339966704249625, 0.006010689073565344, -0.2762567791330412, 0.3332071181431315, 0.1298841749069799, 0.3308514634559807, 0.8263918083397941, -0.39778075890671427, -0.0410702434352641, -0.8381838234718357, -0.524939461711

In [50]:
// compute the conditional means of the model at each time
val log_returns_hat = ARMAForecastOneStep(log_returns, arima4.getARMA())

null
java.lang.NullPointerException
	at Line_53.<init>(Line_53.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [51]:
// compute the residuals = observations - fitted values
var residuals: DoubleArray = DoubleArray(log_returns.size)
for (t in 0 until log_returns.size) {
    residuals[t] = log_returns[t] - log_returns_hat.xHat(t)
}
println("residuals:")
println(Arrays.toString(residuals))

null
java.lang.NullPointerException
	at Line_54.<init>(Line_54.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [52]:
// fit the residuals to a GARCH(1,1) model
val garch_fit = GARCHFit(residuals, 1, 1)
val garch = garch_fit.getModel()
println(String.format("The residual GARCH(1,1) model is: %s", garch))

null
java.lang.NullPointerException
	at Line_55.<init>(Line_55.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [53]:
// fit both the ARMA and GARCH models in one go
val arma_garch_fit = ARMAGARCHFit(
                log_returns, // the input series
                2, // the order for AR in ARMA model
                3, // the order for MA in ARMA model
                1, // the order for GARCH in GARCH model
                1 // the order for ARCH in GARCH model
        )
println("The ARMA model is:")
println(arma_garch_fit.getARMAGARCHModel().getARMAModel())
println("\nThe GARCH model is:")
println(arma_garch_fit.getARMAGARCHModel().getGARCHModel())

null
java.lang.NullPointerException
	at Line_56.<init>(Line_56.jupyter-kts:3)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [54]:
println("multivariate time series")

// construct a bivariate time series
val X_T0 = MultivariateSimpleTimeSeries(
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(-1.875, 1.693),
                        doubleArrayOf(-2.518, -0.03),
                        doubleArrayOf(-3.002, -1.057),
                        doubleArrayOf(-2.454, -1.038),
                        doubleArrayOf(-1.119, -1.086),
                        doubleArrayOf(-0.72, -0.455),
                        doubleArrayOf(-2.738, 0.962),
                        doubleArrayOf(-2.565, 1.992),
                        doubleArrayOf(-4.603, 2.434),
                        doubleArrayOf(-2.689, 2.118)
                )))
println("X_T0: " + X_T0)

multivariate time series
X_T0: [[-1.875000, 1.693000] , [-2.518000, -0.030000] , [-3.002000, -1.057000] , [-2.454000, -1.038000] , [-1.119000, -1.086000] , [-0.720000, -0.455000] , [-2.738000, 0.962000] , [-2.565000, 1.992000] , [-4.603000, 2.434000] , [-2.689000, 2.118000] ]


In [55]:
// first difference of the bivariate time series
val X_T1 = X_T0.diff(1)
println("diff(1): " + X_T1)

diff(1): [[-0.643000, -1.723000] , [-0.484000, -1.027000] , [0.548000, 0.019000] , [1.335000, -0.048000] , [0.399000, 0.631000] , [-2.018000, 1.417000] , [0.173000, 1.030000] , [-2.038000, 0.442000] , [1.914000, -0.316000] ]


In [56]:
// first order lagged time series
val X_T2 = X_T0.lag(1)
println("lag(1): " + X_T2)

lag(1): [[-1.875000, 1.693000] , [-2.518000, -0.030000] , [-3.002000, -1.057000] , [-2.454000, -1.038000] , [-1.119000, -1.086000] , [-0.720000, -0.455000] , [-2.738000, 0.962000] , [-2.565000, 1.992000] , [-4.603000, 2.434000] ]


In [57]:
// drop the first two items
val X_T3 = X_T0.drop(1)
println("drop(1): " + X_T3)

drop(1): [[-2.518000, -0.030000] , [-3.002000, -1.057000] , [-2.454000, -1.038000] , [-1.119000, -1.086000] , [-0.720000, -0.455000] , [-2.738000, 0.962000] , [-2.565000, 1.992000] , [-4.603000, 2.434000] , [-2.689000, 2.118000] ]


In [58]:
println("SPX and APPL")

// read the monthly S&P 500 data from a csv file
val spx_arr = DoubleUtils.readCSV2d("./resources/sp500_monthly.csv", true, true)
// convert the csv file into a matrix for manipulation
val spx_M: Matrix = DenseMatrix(spx_arr)
// extract the column of adjusted close prices
val spx_v: Vector = spx_M.getColumn(5) // adjusted closes

// read the monthly AAPL data from a csv file
val appl_arr = DoubleUtils.readCSV2d("./resources/AAPL_monthly.csv", true, true)
// convert the csv file into a matrix for manipulation
val aapl_M: Matrix = DenseMatrix(appl_arr)
// extract the column of adjusted close prices
val aapl_v: Vector = aapl_M.getColumn(5) // adjusted closes

// combine SPX and AAPL to form a bivariate time series
val mts = MultivariateSimpleTimeSeries(cbind(spx_v, aapl_v))
println("(spx, aapl) prices: \n" + mts)

SPX and APPL


./resources/sp500_monthly.csv (No such file or directory)
java.io.FileNotFoundException: ./resources/sp500_monthly.csv (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:219)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:112)
	at dev.nm.number.DoubleUtils.readCSV2d(wb:1605)
	at dev.nm.number.DoubleUtils.readCSV2d(wb:1591)
	at Line_61.<init>(Line_61.jupyter-kts:4)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.j

In [59]:
// compute the bivariate time series of log returns
val p1: Matrix = mts.drop(1).toMatrix()
val p: Matrix = mts.toMatrix()
var log_returns_M: Matrix = p1.ZERO() // allocate space for the matrix
for (i in 1 until p1.nRows()) { // matrix and vector index starts from 1
    var r: Double = log10(p1.get(i, 1)) - log10(p.get(i, 1))
    log_returns_M.set(i, 1, r)
    r = log10(p1.get(i, 2)) - log10(p.get(i, 2))
    log_returns_M.set(i, 2, r)
}
// convert a matrix to a multivariate time series
val log_returns = MultivariateSimpleTimeSeries(log_returns_M)
println("(spx, aapl) log_returns: \n" + log_returns)

null
java.lang.NullPointerException
	at Line_62.<init>(Line_62.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [60]:
println("VAR, VMA, VARMA, VARIMA and all those")

// construct a VAR(1) model
val PHI: Array<Matrix?> = arrayOfNulls<Matrix>(1)
PHI[0] = DenseMatrix(
        arrayOf(
            doubleArrayOf(0.7, 0.12),
            doubleArrayOf(0.31, 0.6)
        ))
val var1 = VARModel(PHI)
println("unconditional mean = " + var1.unconditionalMean())

VAR, VMA, VARMA, VARIMA and all those
unconditional mean = [0.000000, 0.000000] 


In [61]:
println("VAR models")

// construct a VAR(2) model
val MU: Vector = DenseVector(1.0, 2.0)
val PHI = arrayOf(
    DenseMatrix(
        arrayOf(
            doubleArrayOf(0.2, 0.3),
            doubleArrayOf(0.0, 0.4))),
    DenseMatrix(
        arrayOf(
            doubleArrayOf(0.1, 0.2),
            doubleArrayOf(0.3, 0.1)))
)
val model0 = VARModel(MU, PHI)

// construct a RNG from the model
val sim = VARIMASim(model0)
sim.seed(1234567891L)

// generate a random multivariate time series
val N = 5000
var ts = arrayOfNulls<DoubleArray>(N)
for (i in 0 until N) {
    ts[i] = sim.nextVector()
}
val mts = MultivariateSimpleTimeSeries(DenseMatrix(ts))

// fit the data to a VAR(2) model
val model1 = VARFit(mts, 2)
println("\nμ = ")
println(model1.mu())
println("\nϕ_1 =")
println(model1.AR(1))
println("\nϕ_2 =")
println(model1.AR(2))
println("\nsigma =")
println(model1.sigma())

VAR models

μ = 
[1.074960, 1.980312] 

ϕ_1 =
2x2
	[,1] [,2] 
[1,] 0.214364, 0.275072, 
[2,] -0.005042, 0.408162, 

ϕ_2 =
2x2
	[,1] [,2] 
[1,] 0.106844, 0.197318, 
[2,] 0.293868, 0.106817, 

sigma =
2x2
	[,1] [,2] 
[1,] 1.000000, 0.000000, 
[2,] 0.000000, 1.000000, 


In [62]:
// fit the log returns to an VAR(1) model
val fit = VARFit(log_returns, 1)
println("the estimated phi_1 for var(1) is")
println(fit.AR(1))

null
java.lang.NullPointerException
	at Line_65.<init>(Line_65.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [63]:
/**
* P. J. Brockwell and R. A. Davis, "Example 5.2.1. Chapter 5. Multivariate
* Time Series," in <i>Time Series: Theory and Methods</i>, Springer, 2006.
*/

println("Forecast using the innovation algorithm")

// MA(1) model parameters
val theta: Double = -0.9
val sigma: Double = 1.0

// a multivariate time series (although the dimension is just 1)
val Xt = MultivariateSimpleTimeSeries(
            DenseMatrix(
                arrayOf(
                    doubleArrayOf(-2.58),
                    doubleArrayOf(1.62),
                    doubleArrayOf(-0.96),
                    doubleArrayOf(2.62),
                    doubleArrayOf(-1.36)
                ))
            )

// the autocovariance function
val K = object : MultivariateAutoCovarianceFunction() {

    override fun evaluate(x1: Double, x2: Double): Matrix {
        val i: Int = x1.toInt()
        val j: Int = x2.toInt()

        var k: Double = 0.0

        if (i == j) {
            k = sigma * sigma
            k *= 1 + theta * theta
        }

        if (Math.abs(j - i) == 1) {
            k = theta
            k *= sigma * sigma
        }

        // γ = 0 otherwise
        val result: DenseMatrix = DenseMatrix(arrayOf(doubleArrayOf(k)))
        return result
    }
}

// run the innovation algorithm
val forecast = MultivariateForecastOneStep(Xt, K)

// making forecasts
for (i in 0..5) {
    println(Arrays.toString(forecast.xHat(i).toArray()))
}

Forecast using the innovation algorithm
[0.0]
[1.2828729281767954]
[-0.22269169944446712]
[0.5459302808535396]
[-1.632259252377928]
[-0.22241089478200632]


In [64]:
val log_returns2 = fit.getDemeanedModel() // demeaned version

// predict the future values using the innovation algorithm
val K = VARMAAutoCovariance(log_returns2, log_returns.size())
val forecast = MultivariateForecastOneStep(log_returns, K)
for (i in 0..5) {
    println(Arrays.toString(forecast.xHat(i).toArray()))
}

null
java.lang.NullPointerException
	at Line_67.<init>(Line_67.jupyter-kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:100)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

In [65]:
// construct a VAR(1) model
val THETA: Array<Matrix?> = arrayOfNulls<Matrix>(1)
THETA[0] = DenseMatrix(
        arrayOf(
            doubleArrayOf(0.5, 0.16),
            doubleArrayOf(-0.7, 0.28)
        ))
val vma1 = VMAModel(THETA)
println("unconditional mean = " + vma1.unconditionalMean())

unconditional mean = [0.000000, 0.000000] 


In [66]:
// construct a VARIMA(1,1) model
val PHI1: Array<Matrix?> = arrayOfNulls<Matrix>(1)
PHI1[0] = DenseMatrix(
        arrayOf(
            doubleArrayOf(0.7, 0.0),
            doubleArrayOf(0.0, 0.6)
        ))
val THETA1: Array<Matrix?> = arrayOfNulls<Matrix>(1)
THETA1[0] = DenseMatrix(
        arrayOf(
            doubleArrayOf(0.5, 0.6),
            doubleArrayOf(-0.7, 0.8)
        ))

val varma11: VARMAModel = VARMAModel(
        PHI1,
        THETA1
)
println("unconditional mean = " + varma11.unconditionalMean())

unconditional mean = [0.000000, 0.000000] 


In [67]:
println("autocovariance function for a causal VARMA")

// the AR coefficients
val AR = arrayOf(
    DenseMatrix(
        arrayOf(
            doubleArrayOf(0.5, 0.5),
            doubleArrayOf(0.0, 0.5)))
)

// the MA coefficients
var MA = arrayOf(
    AR[0].t()
)

// SIGMA
val SIGMA: Matrix = DenseMatrix(
                arrayOf(
                    doubleArrayOf(1.0, 0.2),
                    doubleArrayOf(0.2, 1.0)
                ))

// define a VARIMA(1,1) model
val varma11 = VARMAModel(AR, MA, SIGMA)

// comptue the autocovariance function for a VARIMA process up to a certian number of lags
val GAMMA = VARMAAutoCovariance(
                varma11,
                10 // number of lags
        )

// print out the autocovariance function 
for (i in 0..5) {
    println(String.format("GAMMA(%d) = %n", i))
    println(GAMMA.evaluate(1.0))
    println()
}

autocovariance function for a causal VARMA
GAMMA(0) = 

2x2
	[,1] [,2] 
[1,] 5.598667, 3.897333, 
[2,] 3.524000, 3.241333, 

GAMMA(1) = 

2x2
	[,1] [,2] 
[1,] 5.598667, 3.897333, 
[2,] 3.524000, 3.241333, 

GAMMA(2) = 

2x2
	[,1] [,2] 
[1,] 5.598667, 3.897333, 
[2,] 3.524000, 3.241333, 

GAMMA(3) = 

2x2
	[,1] [,2] 
[1,] 5.598667, 3.897333, 
[2,] 3.524000, 3.241333, 

GAMMA(4) = 

2x2
	[,1] [,2] 
[1,] 5.598667, 3.897333, 
[2,] 3.524000, 3.241333, 

GAMMA(5) = 

2x2
	[,1] [,2] 
[1,] 5.598667, 3.897333, 
[2,] 3.524000, 3.241333, 



In [68]:
println("Varim simulation")

// number of random numbers to generate
val T = 100000

// the mean
val MU: Vector = DenseVector(1.0, 1.0)
// the AR coefficients
val PHI = arrayOf(
    DenseMatrix(
    arrayOf(
        doubleArrayOf(0.5, 0.5),
        doubleArrayOf(0.0, 0.5)
)))
// the MA coefficients
val THETA = arrayOf(PHI[0])
// the white noise covariance structure
val SIGMA: Matrix = DenseMatrix(
                arrayOf(
                    doubleArrayOf(1.0, 0.2),
                    doubleArrayOf(0.2, 1.0)
                ))

// construct a VARMA model
val VARMA = VARMAModel(MU, PHI, THETA, SIGMA)

// construct a random number generator from a VARMA model
val SIM = VARIMASim(VARMA)
SIM.seed(1234567890L)

// produce the random vectors
val data = arrayOfNulls<DoubleArray>(T)
for (i in 0 until T) {
    data[i] = SIM.nextVector()
}

// statistics about the simulation
// println(String.format("each vector size = %d%n", data[0].size()))
// println(String.format("sample size = %d%n", data.length))

// compute the theoretical mean of the data
// val theoretical_mean: DoubleArray = Inverse(DenseMatrix(arrayOf(doubleArrayOf(2.0, 2.0))).ONE().minus(VARMA.AR(1))).multiply(DenseVector(MU)).toArray()
// println("theoretical mean =")
// println(Arrays.toString(theoretical_mean))

// cast the random data in matrix form for easy manipulation
val dataM: Matrix = MatrixFactory.rbind(DenseMatrix(data))
// compute the sample mean of the data
val sample_mean1: Double = Mean(dataM.getColumn(1).toArray()).value()
val sample_mean2: Double = Mean(dataM.getColumn(2).toArray()).value()
println(String.format("Sample mean of the first variable = %f%n", sample_mean1))
println(String.format("Sample mean of the second variable = %f%n", sample_mean2))

// compute the theoretical covariance of the data
val cov_theoretical: Matrix = DenseMatrix(
        arrayOf(
            doubleArrayOf(811.0 / 135.0, 101.0 / 45.0),
            doubleArrayOf(101.0 / 45.0, 7.0 / 3.0)
        ))
println("Theoretical covariance =")
println(cov_theoretical)

// compute the sample covariance of the data
val sample_cov = SampleCovariance(dataM)
println("\nSample covariance =")
println(sample_cov)

Varim simulation
Sample mean of the first variable = 3.994879

Sample mean of the second variable = 2.005700

Theoretical covariance =
2x2
	[,1] [,2] 
[1,] 6.007407, 2.244444, 
[2,] 2.244444, 2.333333, 

Sample covariance =
2x2
	[,1] [,2] 
[1,] 5.315639, 1.790519, 
[2,] 1.790519, 2.340646, 


In [69]:
println("convert VAR to VECM")

// define a VAR(2) model
val varxModel = VARXModel(
        arrayOf(
            DenseMatrix(
                    arrayOf(
                        doubleArrayOf(-0.210, 0.167),
                        doubleArrayOf(0.512, 0.220)
                    )),
            DenseMatrix(
                    arrayOf(
                        doubleArrayOf(0.743, 0.746),
                        doubleArrayOf(-0.405, 0.572)
                    ))
        ),
        null)

// construct a VECM fron a VAR
val vecm = VECMTransitory(varxModel)

println("dimension = " + vecm.dimension())
println("PI, the impact matrix = ")
println(vecm.pi())
println("GAMMA = ")
println(vecm.gamma(1))

convert VAR to VECM
dimension = 2
PI, the impact matrix = 
2x2
	[,1] [,2] 
[1,] -0.467000, 0.913000, 
[2,] 0.107000, -0.208000, 
GAMMA = 
2x2
	[,1] [,2] 
[1,] -0.743000, -0.746000, 
[2,] 0.405000, -0.572000, 


In [70]:
println("cointegration between SPX and AAPL")

// read the monthly S&P 500 data from a csv file
val spx_arr = DoubleUtils.readCSV2d("./resources/sp500_monthly.csv", true, true)

// convert the csv file into a matrix for manipulation
val spx_M1: Matrix = DenseMatrix(spx_arr)
// remove the data before 2008
val spx_M2: Matrix = MatrixFactory.subMatrix(spx_M1, 325, spx_M1.nRows(), 1, spx_M1.nCols())
// extract the column of adjusted close prices
val spx_v: Vector = spx_M2.getColumn(5) // adjusted closes

// read the monthly AAPL data from a csv file
val appl_arr = DoubleUtils.readCSV2d("./resources/AAPL_monthly.csv", true, true)
// convert the csv file into a matrix for manipulation
val aapl_M1: Matrix = DenseMatrix(appl_arr)
// remove the data before 2008
val aapl_M2: Matrix = MatrixFactory.subMatrix(aapl_M1, 325, aapl_M1.nRows(), 1, aapl_M1.nCols())
// extract the column of adjusted close prices
val aapl_v: Vector = aapl_M2.getColumn(5) // adjusted closes

// combine SPX and AAPL to form a bivariate time series
val mts = MultivariateSimpleTimeSeries(cbind(spx_v, aapl_v))
//      println("(spx, aapl) prices: \n" + mts)

// run cointegration on all combinations of available test and trend types
for (test: Test in Test.values()) {
    for (trend: dev.nm.stat.cointegration.JohansenAsymptoticDistribution.TrendType in dev.nm.stat.cointegration.JohansenAsymptoticDistribution.TrendType.values()) {
        val coint = CointegrationMLE(mts, true)
        val johansen = JohansenTest(test, trend, coint.getEigenvalues().size())
        println("alpha:")
        println(coint.alpha())
        println("beta:")
        println(coint.beta())
        println("Johansen test: "
                + test.toString()
                + "\t" + trend.toString()
                + "\t eigenvalues: " + coint.getEigenvalues()
                + "\t statistics: " + johansen.getStats(coint)
        )
        println("")
    }
}

// run ADF test to check if the cointegrated series is indeed stationary
val betas: DoubleArray = doubleArrayOf(-12.673296, -6.995392)
for (beta: Double in betas) {
    println(String.format("testing for beta = %f%n", beta))
    val ci: Vector = spx_v.add(aapl_v.scaled(beta))
    val adf = AugmentedDickeyFuller(
                    ci.toArray(),
                    TrendType.CONSTANT, // constant drift term
                    4, // the lag order
                    null
            )

    println("H0: " + adf.getNullHypothesis())
    println("H1: " + adf.getAlternativeHypothesis())
    println(String.format("the p-value for the test = %f%n", adf.pValue()))
    println(String.format("the statistics for the test = %f%n", adf.statistics()))
    println("")
}

cointegration between SPX and AAPL


./resources/sp500_monthly.csv (No such file or directory)
java.io.FileNotFoundException: ./resources/sp500_monthly.csv (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:219)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:112)
	at dev.nm.number.DoubleUtils.readCSV2d(wb:1605)
	at dev.nm.number.DoubleUtils.readCSV2d(wb:1591)
	at Line_73.<init>(Line_73.jupyter-kts:4)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.j